In [167]:
import requests
import json
import numpy as np
import pandas as pd
import pprint
import math
import datetime
pd.options.display.max_columns = 999
# np.set_printoptions(suppress=True)
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
shopify_data = pd.read_csv("pcsg_shopify_20180531.csv", low_memory=False, dtype={'Lineitem sku': 'str'})

def try_float(x):
    try:
        return str(format(float(x), '.0f'))
    except ValueError as e:
        return x

shopify_data['Name'] = shopify_data['Name'].apply(try_float)
sku_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx", sheetname='SKU List' )
bundle_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx",
                              sheetname='Bundles' ).rename(columns = {'Parent SKU': "parent_sku",
                                                                      'Parent Name': 'parent_name',
                                                                      'Quantity': 'child_quantity',
                                                                      'Child SKU': "child_sku",
                                                                      'Child Name': 'child_name'}).astype({'child_sku': 'str',
                                                                                                           'parent_sku': 'str'})

In [168]:
shopify_data_merge = pd.merge(shopify_data, bundle_master, left_on="Lineitem sku", right_on = 'parent_sku', how='left')
shopify_data_merge#[shopify_data_merge['Lineitem sku'] == '4707']

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,parent_sku,parent_name,child_sku,child_name,child_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price
0,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.000,0.000,0.000,42.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.000,nan,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.000,Paula's Choice,0.000,NaN,NaN,nan,494491000000.000,NaN,Low,web,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4707,Clear Regular Strength Trial Kit,6006,30 ml,1.000,8.000,8.000,39.000,29.000,0.256,5.949
1,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.000,0.000,0.000,42.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.000,nan,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.000,Paula's Choice,0.000,NaN,NaN,nan,494491000000.000,NaN,Low,web,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4707,Clear Regular Strength Trial Kit,6107,14 ml,1.000,14.000,14.000,39.000,29.000,0.256,10.410
2,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.000,0.000,0.000,42.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.000,nan,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.000,Paula's Choice,0.000,NaN,NaN,nan,494491000000.000,NaN,Low,web,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4707,Clear Regular Strength Trial Kit,6207,30 ml,1.000,17.000,17.000,39.000,29.000,0.256,12.641
3,191570000000,jnbitalac@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,2018-05-31 23:59:17 +0800,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,13.000,nan,2016,True,False,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,Paula's Choice,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan
4,191570000000,isjanelle@gmail.com,paid,2018-05-31 23:54:52 +0800,unfulfilled,NaN,yes,SGD,68.000,0.000,0.000,68.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:54:52 +0800,1,C15 Super Bo

In [82]:
shopify_data_merge['parent_sku'].fillna(shopify_data_merge['Lineitem sku'], inplace=True)
shopify_data_merge['child_sku'].fillna(shopify_data_merge['Lineitem sku'], inplace=True)
shopify_data_merge['child_quantity'].fillna(shopify_data_merge['Lineitem quantity'], inplace=True)
shopify_data_merge['Discount Unit Price'].fillna(shopify_data_merge['Lineitem price'], inplace=True)
# shopify_data_merge['Unit Price'].fillna(shopify_data_merge['Lineitem compare at price'], inplace=True)
# shopify_data_merge['total_quantity'].fillna(shopify_data_merge['Lineitem quantity'], inplace = True)
shopify_data_merge['child_subtotal_quantity'] =  shopify_data_merge['child_quantity'] * shopify_data_merge['Lineitem quantity']
shopify_data_merge['child_subtotal'] =  shopify_data_merge['child_subtotal_quantity'] * shopify_data_merge['Discount Unit Price']
shopify_data_final = shopify_data_merge.copy()


In [83]:
# shopify_data_final.to_excel('formatted_shopify_data.xlsx')
shopify_data_final

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,parent_sku,parent_name,child_sku,child_name,child_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price,child_subtotal_quantity,child_subtotal
0,1.9157E+11,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.000,0.000,0.000,42.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.000,nan,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.000,Paula's Choice,0.000,NaN,NaN,nan,494491000000.000,NaN,Low,web,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4707,Clear Regular Strength Trial Kit,6006,30 ml,1.000,8.000,8.000,39.000,29.000,0.256,5.949,1.000,5.949
1,1.9157E+11,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.000,0.000,0.000,42.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.000,nan,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.000,Paula's Choice,0.000,NaN,NaN,nan,494491000000.000,NaN,Low,web,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4707,Clear Regular Strength Trial Kit,6107,14 ml,1.000,14.000,14.000,39.000,29.000,0.256,10.410,1.000,10.410
2,1.9157E+11,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.000,0.000,0.000,42.000,NaN,0.000,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.000,nan,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.000,Paula's Choice,0.000,NaN,NaN,nan,494491000000.000,NaN,Low,web,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4707,Clear Regular Strength Trial Kit,6207,30 ml,1.000,17.000,17.000,39.000,29.000,0.256,12.641,1.000,12.641
3,1.9157E+11,jnbitalac@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,2018-05-31 23:59:17 +0800,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,13.000,nan,2016,True,False,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,Paula's Choice,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2016,NaN,2016,NaN,1.000,nan,nan,nan,nan,nan,13.000,1.000,13.000
4,1.9157E+11,isjanelle@gmail.com,paid,2018-05-31 23:54:52 +0800,unfulfilled,NaN,yes,SGD,68.000,0.000,0.000,6

In [85]:
shopify_data_final.groupby(['Name'])['child_subtotal'].sum()

Name            Created at               
#1002           2016-10-04 18:58:07 +0800   112.000
#1003           2016-10-12 14:34:46 +0800    43.000
#1004           2016-10-12 14:48:15 +0800     1.500
1.91568E+11     2017-09-14 19:36:57 +0800   138.000
                2017-09-14 19:40:54 +0800   163.000
                2017-09-14 19:58:02 +0800   222.000
                2017-09-14 21:14:18 +0800   233.000
                2017-09-14 21:59:29 +0800    34.000
                2017-09-14 22:36:44 +0800   207.000
                2017-09-14 22:41:26 +0800    43.000
                2017-09-14 22:43:36 +0800    30.000
                2017-09-14 22:46:09 +0800    43.000
                2017-09-14 23:44:25 +0800   126.000
                2017-09-15 00:19:21 +0800   116.000
                2017-09-15 00:20:09 +0800   111.000
                2017-09-15 01:06:24 +0800   811.000
                2017-09-15 02:26:17 +0800   170.000
                2017-09-15 10:24:41 +0800    29.000
                2017-0